In [1]:
import random
import copy
import time
import pandas as pd
import numpy as np
import gc
import re
import torch

#import spacy
from tqdm import tqdm_notebook, tnrange
from tqdm.auto import tqdm

tqdm.pandas(desc='Progress')
from collections import Counter

from nltk import word_tokenize

import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
from torch.utils.data import Dataset, DataLoader
from torch.nn.utils.rnn import pack_padded_sequence, pad_packed_sequence
from torch.autograd import Variable
from sklearn.metrics import f1_score
import os 

from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences # There's a PyTorch implementation but for Tensors.

# cross validation and metrics
from sklearn.model_selection import StratifiedKFold
from sklearn.metrics import f1_score
from torch.optim.optimizer import Optimizer

from sklearn.preprocessing import StandardScaler
from multiprocessing import  Pool
from functools import partial
import numpy as np
from sklearn.decomposition import PCA
import torch as t
import torch.nn as nn
import torch.nn.functional as F

from dataclasses import dataclass

import matplotlib.pyplot as plt

import nltk
nltk.download('punkt')
nltk.download('stopwords')
nltk.download('wordnet')

/opt/conda/lib/python3.8/site-packages/tqdm/std.py:725: FutureWarning: The Panel class is removed from pandas. Accessing it from the top-level namespace will also be removed in the next version
  from pandas import Panel
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


True

In [2]:
embed_size = 300 # how big is each word vector
max_features = 40000 # how many unique words to use (i.e num rows in embedding vector)
maxlen = 750 # max number of words in a question to use
batch_size = 128 # how many samples to process at once
n_epochs = 5 # how many times to iterate over all samples
n_splits = 5 # Number of K-fold Splits
SEED = 10
debug = 0

### load processed data

In [3]:
import pandas as pd

df = pd.read_csv('mbti_rm_stop_lemmatized.csv', index_col=None) 
df.head()

,type,posts_length,lemmatized
0,INFJ,304,enfp intj moments sportscenter top ten play pr...
1,ENTP,554,find lack post alarm sex bore position often e...
2,INTP,427,good one course say know bless curse absolutel...
3,INTJ,521,dear intp enjoy conversation day esoteric gabb...
4,ENTJ,477,fire another silly misconception approach logi...


### encode labels

In [4]:
int2mbti={0:'ENFJ',1:'ENFP',2:'ENTJ',3:'ENTP',4:'ESFJ',5:'ESFP',6:'ESTJ',7:'ESTP',8:'INFJ',9:'INFP',10:'INTJ',11:'INTP',12:'ISFJ',13:'ISFP',14:'ISTJ',15:'ISTP'}
mbti2int={'ENFJ':0,'ENFP':1,'ENTJ':2,'ENTP':3,'ESFJ':4,'ESFP':5,'ESTJ':6,'ESTP':7,'INFJ':8,'INFP':9,'INTJ':10,'INTP':11,'ISFJ':12,'ISFP':13,'ISTJ':14,'ISTP':15}

In [5]:
labels = df.type.tolist()
labels = [mbti2int.get(label) for label in labels]
print(labels[-50:])

[11, 10, 1, 15, 11, 13, 9, 14, 3, 9, 8, 9, 10, 9, 3, 11, 9, 8, 3, 8, 8, 9, 11, 9, 9, 8, 15, 12, 9, 14, 8, 8, 10, 4, 1, 9, 3, 10, 11, 11, 3, 10, 3, 10, 8, 13, 1, 11, 9, 9]


### clean posts

In [6]:
!pip install autocorrect

In [7]:
import nltk
from nltk.corpus import stopwords
import re
from nltk.tokenize import word_tokenize 
from nltk.stem import WordNetLemmatizer 
from autocorrect import Speller 

nltk.download('wordnet')
nltk.download('stopwords')

class data_preprocessing():
    
    def remove_links(text):
        remove_https = re.sub(r'http\S+', '', text)
        remove_com = re.sub(r"\ [A-Za-z]*\.com", " ", remove_https)
        return remove_com
    
    def remove_digits(text):
        return re.sub(r'\d+', ' ', text)
    
    def remove_symbols(text):
        REPLACE_BY_SPACE_RE = re.compile('[/(){}\[\]\|@,;]')
        BAD_SYMBOLS_RE = re.compile('(@\[A-Za-z0-9]+)|([^0-9A-Za-z \t])|(\w+:\/\/\S+)')
        t = REPLACE_BY_SPACE_RE.sub(' ', text)
        t = BAD_SYMBOLS_RE.sub(' ', t)
        return t
    
    def deduce_repeated_characters(text):
        Pattern_alpha = re.compile(r"([A-Za-z])\1{1,}", re.DOTALL)
        Formatted_text = Pattern_alpha.sub(r"\1\1", text) 
        Pattern_Punct = re.compile(r'([.,/#!$%^&*?;:{}=_`~()+-])\1{1,}')
        Combined_Formatted = Pattern_Punct.sub(r'\1', Formatted_text)
        Final_Formatted = re.sub(' {2,}',' ', Combined_Formatted)
        return Final_Formatted
    
    def remove_special_characters(text):
        return re.sub(r"[^a-zA-Z0-9:$-,%.?!]+", ' ', text)
    
    def spelling_correction(text):
        spell = Speller(lang='en')
        Corrected_text = spell(text)
        return Corrected_text
    
    def lemmatization(text):
        w_tokenizer = nltk.tokenize.WhitespaceTokenizer()
        lemmatizer = nltk.stem.WordNetLemmatizer()
        lemma = [lemmatizer.lemmatize(w,'v') for w in w_tokenizer.tokenize(text)]
        return lemma

[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [8]:
STOPWORDS = set(stopwords.words('english'))

def clean_text(t):
    t = t.lower()
    t = t.replace("|||"," ")   
    t = data_preprocessing.remove_links(t)
    t = data_preprocessing.remove_digits(t)
    t = data_preprocessing.remove_symbols(t)
    t = data_preprocessing.deduce_repeated_characters(t)
    t = data_preprocessing.remove_special_characters(t)
#         t = data_preprocessing.spelling_correction(t)
    t = data_preprocessing.lemmatization(t)
    t = ' '.join(word for word in t.split() if word not in STOPWORDS) 
    return t

In [9]:
from sklearn.model_selection import train_test_split
train_X, test_X, train_y, test_y = train_test_split(df['lemmatized'],
                                                    df['type'],
                                                    stratify=df['type'], 
                                                    test_size=0.25)

In [10]:
## Tokenize the sentences
tokenizer = Tokenizer(num_words=max_features)
tokenizer.fit_on_texts(list(train_X))
train_X = tokenizer.texts_to_sequences(train_X)
test_X = tokenizer.texts_to_sequences(test_X)

## Pad the sentences 
x_train = pad_sequences(train_X)
x_test = pad_sequences(test_X)

In [11]:
from sklearn.preprocessing import LabelEncoder
le = LabelEncoder()
y_train = le.fit_transform(train_y.values)
y_test = le.transform(test_y.values)

In [12]:
def load_fasttext(word_index):    
    EMBEDDING_FILE = 'wiki-news-300d-1M.vec'
    def get_coefs(word,*arr): return word, np.asarray(arr, dtype='float32')
    embeddings_index = dict(get_coefs(*o.split(" ")) for o in open(EMBEDDING_FILE) if len(o)>100 and o.split(" ")[0] in word_index )

    all_embs = np.stack(embeddings_index.values())
    emb_mean,emb_std = all_embs.mean(), all_embs.std()
    embed_size = all_embs.shape[1]

    embedding_matrix = np.random.normal(emb_mean, emb_std, (max_features, embed_size))
    for word, i in word_index.items():
        if i >= max_features: continue
        embedding_vector = embeddings_index.get(word)
        if embedding_vector is not None: embedding_matrix[i] = embedding_vector

    return embedding_matrix

In [13]:
embedding_matrix = load_fasttext(tokenizer.word_index)

/opt/conda/lib/python3.8/site-packages/IPython/core/interactiveshell.py:3427: FutureWarning: arrays to stack must be passed as a "sequence" type such as list or tuple. Support for non-sequence iterables such as generators is deprecated as of NumPy 1.16 and will raise an error in the future.
  exec(code_obj, self.user_global_ns, self.user_ns)


### create datasets

In [14]:
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.nn.utils.rnn import pad_sequence

In [15]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
vocab_size = 40000
seq_len = 500
num_labels = 16
EMBEDDING_DIM=embed_size

# device = torch.device('cpu')
print("deive type: ",device)

deive type:  cuda


In [16]:
def collate_batch(batch):
    label_list, text_list = [], []
    for (_post, _label) in batch:
        label_list.append(_label)
        text_list.append(_post)
    # label must be in the same size as target
    label_list = torch.tensor(label_list)
    text_list = torch.stack(text_list)
    return text_list.to(device), label_list.to(device)

In [17]:
from torch.utils.data import TensorDataset, DataLoader
# # create Tensor datasets
train_data = TensorDataset(torch.from_numpy(x_train).to(device), torch.tensor(y_train).to(device))
test_data = TensorDataset(torch.from_numpy(x_test).to(device),torch.tensor(y_test).to(device))
# val_data = TensorDataset(torch.from_numpy(x_val).to(device),torch.tensor(y_val).to(device))

# # dataloaders
batch_size = 256

# # make sure to SHUFFLE your data
train_loader = DataLoader(train_data, shuffle=True, batch_size=batch_size)
test_loader = DataLoader(test_data, shuffle=False, batch_size=batch_size)
# val_loader = DataLoader(val_data, shuffle=False, batch_size=batch_size, collate_fn=collate_batch)

### LSTM model

In [18]:
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.nn.utils.rnn import pad_sequence

In [19]:
class SimpleAttention(nn.Module):

    def __init__(self, input_dim):
        super(SimpleAttention, self).__init__()
        self.input_dim = input_dim
        self.scalar = nn.Linear(self.input_dim,num_labels,bias=False)

    def forward(self, M, x=None):
        """
        M -> (seq_len, batch, vector)
        x -> dummy argument for the compatibility with MatchingAttention
        """
        scale = self.scalar(M) # seq_len, batch, 1
        alpha = F.softmax(scale, dim=0).permute(1,2,0) # batch, 1, seq_len
        attn_pool = torch.bmm(alpha, M.transpose(0,1))[:,0,:] # batch, vector

        return attn_pool, alpha

In [20]:
class LSTMcustom(nn.Module):
    def __init__(self):
        super().__init__()
        # embeddingbag outputs the average of all the words in a sentence
        self.embedding = nn.Embedding(max_features, embed_size)
        self.embedding.weight = nn.Parameter(torch.tensor(embedding_matrix, dtype=torch.float32))
        self.embedding.weight.requires_grad = False
#         self.cnn = torch.nn.Conv1d(embed_size,20,2)
        self.lstm = nn.LSTM(embed_size, 200, 1, bidirectional=False, batch_first = True)      
        self.attention = SimpleAttention(200)  
        self.linears = nn.Sequential(
            nn.Linear(200, 100),
            nn.ReLU(),
            nn.Dropout(0.25),
            nn.Linear(100, num_labels),
#             nn.Softmax()
        )
        
                
    def forward(self, text):
        embedded = self.embedding(text) # (batch_size, sent_len, emb_size)

        lstm_out,_ = self.lstm(embedded) 
        lstm_out = lstm_out.permute(1,0,2)
        
        atten_out, alpha = self.attention(lstm_out)       
        
#         return torch.log_softmax(torch.tanh(self.linears(atten_out)),1)
        return torch.log_softmax(self.linears(atten_out),1)

In [21]:
import time

def train(dataloader):
#     model.train()
    total_acc, total_count = 0, 0
    log_interval = 20
    start_time = time.time()

    for idx, (text, label) in enumerate(dataloader):
        model.train()
        optimizer.zero_grad()
        # forward propagation
        predicted_label = model(text)

        # calculate loss and backpropagate to model paramters
        loss = criterion(predicted_label, label)
        loss.backward()
        torch.nn.utils.clip_grad_norm_(model.parameters(), 0.1)
        
        # update parameters by stepping the optimizer
        optimizer.step()
        predicted_label = torch.argmax(predicted_label,1)
        total_acc += (predicted_label == label).sum().item()
        total_count += label.size(0)
        if idx % log_interval == 0 and idx > 0:
            elapsed = time.time() - start_time
            print('| epoch {:3d} | {:5d}/{:5d} batches '
                  '| accuracy {:8.3f}| loss {:8f}'.format(epoch, idx, len(dataloader),
                                              total_acc/total_count,loss.item()))
            total_acc, total_count = 0, 0
            start_time = time.time()


def evaluate(dataloader):
    model.eval()
    total_acc, total_count = 0, 0

    with torch.no_grad():
        for idx, (text, label) in enumerate(dataloader):
            predicted_label = model(text)
            loss = criterion(predicted_label, label)
            total_acc += (predicted_label.argmax(1) == label).sum().item()
            total_count += label.size(0)
#     print('val accuracy {:8.2f} | val loss {:8f}'.format(total_acc/total_count,loss.item()))
    return total_acc/total_count

In [24]:
# Hyperparameters
EPOCHS = 15 # epoch

model = LSTMcustom().to(device)
optimizer = torch.optim.Adam(filter(lambda p: p.requires_grad, model.parameters()), lr=0.01)
criterion = nn.CrossEntropyLoss().to(device)
total_accu = None

for epoch in range(1, EPOCHS + 1):
    epoch_start_time = time.time()
    train(train_loader)

| epoch   1 |    20/   26 batches | accuracy    0.190| loss 2.362297
| epoch   2 |    20/   26 batches | accuracy    0.205| loss 2.312289
| epoch   3 |    20/   26 batches | accuracy    0.207| loss 2.345776
| epoch   4 |    20/   26 batches | accuracy    0.291| loss 2.098326
| epoch   5 |    20/   26 batches | accuracy    0.395| loss 1.872772
| epoch   6 |    20/   26 batches | accuracy    0.509| loss 1.591138
| epoch   7 |    20/   26 batches | accuracy    0.608| loss 1.349994
| epoch   8 |    20/   26 batches | accuracy    0.635| loss 1.312217
| epoch   9 |    20/   26 batches | accuracy    0.675| loss 1.217699
| epoch  10 |    20/   26 batches | accuracy    0.706| loss 1.121703
| epoch  11 |    20/   26 batches | accuracy    0.749| loss 0.909326
| epoch  12 |    20/   26 batches | accuracy    0.775| loss 0.815796
| epoch  13 |    20/   26 batches | accuracy    0.794| loss 0.763319
| epoch  14 |    20/   26 batches | accuracy    0.828| loss 0.611910
| epoch  15 |    20/   26 batches 

In [25]:
accu_test = evaluate(test_loader)
print('test accuracy {:8.2f}%'.format(accu_test))

test accuracy     0.63%
